In [ ]:
%matplotlib inline 

import pandas as pd
import numpy as np
from sklearn import cluster, metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet, fcluster
from scipy.spatial.distance import pdist
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot') 

## 1. Format the Data

In [ ]:
ap = pd.read_csv('../../assets/datasets/airport2.csv')
ap.head()

Convert the class labels, in this case the airports, to numeric. *Remember*, we've truncated the airport dataset for simplicty to make this part easy! 

In [ ]:
def air_to_numeric(x):
    if x=='ABQ':
        return 1
    if x=='ANC':
        return 2
    if x=='ATL':
        return 3

In [ ]:
ap['Airport'] = ap['Airport'].apply(air_to_numeric)

## 2. Plot the data

In [ ]:
ap.plot(kind='scatter',x='Departure Cancellations',y='Arrival Cancellations')

In [ ]:
ap.plot(kind='scatter',x='Departure Cancellations',y='Departure Diversions')

**Question**: Based on the graphs, how many clusters do you estimate that we have?

## 3.  Perform a K-Means Clustering 

#### 3.1 Split the dataset into the attribute data and class labels

In [ ]:
x = ap.ix[:,1:5]
y = ap.ix[:,0]

In [ ]:
X = x.as_matrix(columns=None)

#### 3.2 Conduct the k-means clustering

In [ ]:
k = 3
kmeans = cluster.KMeans(n_clusters=k)
kmeans.fit(X)

#### 3.3 Compute the labels and centroids

In [ ]:
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

In [ ]:
print(labels)
print(centroids)

#### 3.4 Format the resulting predicted "y" variables and check the accuracy score, classification report, and confusion matrix

In [ ]:
metrics.accuracy_score(y, labels)

In [ ]:
print(metrics.classification_report(y, labels))

In [ ]:
print(metrics.confusion_matrix(y, labels))

**Question**: What are we understand from this these performance metrics? 

## 4. Extra Trees Classifier

In [ ]:
trees = ExtraTreesClassifier()
trees.fit(X, labels)

In [ ]:
print(trees.feature_importances_)

## 5. KNN Classifier

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X, labels)

In [ ]:
exp = labels
predicted = knn.predict(x)

In [ ]:
print(metrics.classification_report(exp, predicted))
print(metrics.confusion_matrix(exp, predicted))

## 6. Hierarchical Clustering

#### 1. Create the linkage for clustering

In [ ]:
Z = linkage(X, 'ward')

In [ ]:
c, coph_dists = cophenet(Z, pdist(X))
c

In [ ]:
plt.figure(figsize=(30, 10))
plt.title('Dendrogram')
plt.xlabel('Index Numbers')
plt.ylabel('Distance')
dendrogram(
    Z,
    leaf_rotation=90.,  
    leaf_font_size=8.,
)
plt.show()

In [ ]:
plt.title('Truncated Dendrogram')
plt.xlabel('Index Numbers')
plt.ylabel('Distance')
dendrogram(
    Z,
    truncate_mode='lastp',  
    p=15,  
    show_leaf_counts=False,  
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,  
)
plt.show()

In [ ]:
max_d = 30
clusters = fcluster(Z, max_d, criterion='distance')
clusters

In [ ]:
plt.scatter(X[:,0], X[:,2], c=clusters, cmap='prism')
plt.show()

## Extra Trees Classifier

In [ ]:
trees2 = ExtraTreesClassifier()
trees2.fit(X, clusters)

In [ ]:
print(trees2.feature_importances_)

## KNN Classifier

In [ ]:
knn2 = KNeighborsClassifier()
knn2.fit(X, clusters)

In [ ]:
exp2 = clusters
predicted2 = knn2.predict(X)

In [ ]:
print(metrics.classification_report(exp, predicted))
print(metrics.confusion_matrix(exp, predicted))